In [1]:
!wget https://ws-assets-prod-iad-r-iad-ed304a55c2ca1aee.s3.us-east-1.amazonaws.com/508f04dc-026a-4260-ac15-c6f011d50cf9/agentcore-multi-target-gateway-starter-code.zip
!unzip -o agentcore-multi-target-gateway-starter-code.zip
!ls -la

--2025-11-01 13:38:59--  https://ws-assets-prod-iad-r-iad-ed304a55c2ca1aee.s3.us-east-1.amazonaws.com/508f04dc-026a-4260-ac15-c6f011d50cf9/agentcore-multi-target-gateway-starter-code.zip
Resolving ws-assets-prod-iad-r-iad-ed304a55c2ca1aee.s3.us-east-1.amazonaws.com (ws-assets-prod-iad-r-iad-ed304a55c2ca1aee.s3.us-east-1.amazonaws.com)... 52.217.68.72, 16.15.188.252, 52.216.214.138, ...
Connecting to ws-assets-prod-iad-r-iad-ed304a55c2ca1aee.s3.us-east-1.amazonaws.com (ws-assets-prod-iad-r-iad-ed304a55c2ca1aee.s3.us-east-1.amazonaws.com)|52.217.68.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45432 (44K) [application/zip]
Saving to: ‘agentcore-multi-target-gateway-starter-code.zip’

agentcore-multi-tar 100%[===================>]  44.37K  --.-KB/s    in 0.001s  

2025-11-01 13:38:59 (58.3 MB/s) - ‘agentcore-multi-target-gateway-starter-code.zip’ saved [45432/45432]

Archive:  agentcore-multi-target-gateway-starter-code.zip
   creating: agentcore-gateway-sta

In [2]:
!pip install --force-reinstall -U -r requirements.txt --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.4.0 requires nvidia-ml-py3<8.0,>=7.352.0, which is not installed.
dash 2.18.1 requires dash-core-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-html-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-table==5.0.0, which is not installed.
jupyter-ai 2.31.6 requires faiss-cpu!=1.8.0.post0,<2.0.0,>=1.8.0, which is not installed.
aiobotocore 2.21.1 requires botocore<1.37.2,>=1.37.0, but you have botocore 1.40.64 which is incompatible.
autogluon-multimodal 1.4.0 requires jsonschema<4.24,>=4.18, but you have jsonschema 4.25.1 which is incompatible.
autogluon-multimodal 1.4.0 requires transformers[sentencepiece]<4.50,>=4.38.0, but you have transformers 4.55.2 which is incompatible.
autogluon-timeseries 1.4.0 requires transformers[sentencepiece]<4.50,>=4.38.0,

In [3]:
# Set AWS credentials if not using Amazon SageMaker notebook
import os
# os.environ['AWS_ACCESS_KEY_ID'] = '' # Set the access key
# os.environ['AWS_SECRET_ACCESS_KEY'] = '' # Set the secret key
os.environ['AWS_DEFAULT_REGION'] = os.environ.get('AWS_REGION', 'us-east-1') # set the AWS region

In [7]:
import os
import sys
import utilsforecast

# Get the directory of the current script
if '__file__' in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
else:
    current_dir = os.getcwd()  # Fallback if __file__ is not defined (e.g., Jupyter)

# Navigate to the directory containing utils.py (one level up)
utils_dir = os.path.abspath(os.path.join(current_dir, '..'))

# Add to sys.path
sys.path.insert(0, utils_dir)

# Now you can import utils


In [10]:
import json
import time

def create_lambda_execution_role():
    role_name = "gateway_lambda_iamrole"
    
    trust_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }
    
    try:
        response = iam.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=json.dumps(trust_policy),
            Description="Role for Lambda functions in AgentCore Gateway"
        )
        
        # Attach basic Lambda execution policy
        iam.attach_role_policy(
            RoleName=role_name,
            PolicyArn="arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole"
        )
        
        return response['Role']['Arn']
        
    except iam.exceptions.EntityAlreadyExistsException:
        response = iam.get_role(RoleName=role_name)
        return response['Role']['Arn']

# Create the Lambda execution role
lambda_role_arn = create_lambda_execution_role()
print(f"Lambda role ARN: {lambda_role_arn}")

# Wait for role to propagate
print("Waiting for role to propagate...")
time.sleep(10)

NameError: name 'iam' is not defined